<a href="https://colab.research.google.com/gist/zredlined/3091bc76da6a43654e43abe5c46a799a/batch-prompting-llama-2-with-gretel-gpt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


<br>

<center><a href=https://gretel.ai/><img src="https://assets-global.website-files.com/5ec4696a9b6d337d51632638/651c817db814b3623bc572e2_DES-380%20Prompting%20Llama%202.jpg" alt="Gretel" width="350"/></a></center>

<br>

##Prompting Llama-2 at Scale with Gretel

Discover how to efficiently use Gretel's platform for prompting Llama-2 on large datasets, whether you're completing answers, generating synthetic text, or labeling.

In this Blueprint, we will leverage Gretel's platform to **prompt** a Llama-2-7B parameter large language model (LLM) in batch mode. If you already worked through the tabular Blueprints, you will notice that Gretel's SDK interface is the same for all data modalities, so you only need to learn it once!

As with the previous Blueprints, we will submit training and generation jobs to the Gretel Cloud, which will spin up the compute resources required for fine tuning and prompting an LLM.

## In the right place?

If you are new to Gretel, we recommend starting with these [SDK Blueprints](https://github.com/gretelai/gretel-blueprints/tree/main/sdk_blueprints):

1. [Gretel 101 Blueprint](https://colab.research.google.com/github/gretelai/gretel-blueprints/blob/main/sdk_blueprints/Gretel_101_Blueprint.ipynb).

2. [Gretel Advanced Tabular Blueprint](https://colab.research.google.com/github/gretelai/gretel-blueprints/blob/main/sdk_blueprints/Gretel_Advanced_Tabular_Blueprint.ipynb)

**Note:** You will need a [free Gretel account](https://console.gretel.ai/) to run this notebook.

<br>

#### Ready? Let's go 🚀

In [ ]:
!pip install -Uqq gretel-client datasets

## 🛜 Configure your Gretel session

- Each `Gretel` instance is bound to a single [Gretel project](https://docs.gretel.ai/guides/gretel-fundamentals/projects).  

- You can set the project name at instantiation, or you can use the `set_project` method.

- If you do not set the project, a random project will be created with your first job submission.


- You can retrieve your API key [here](https://console.gretel.ai/users/me/key).

In [ ]:
from gretel_client import Gretel

gretel = Gretel(project_name="text-gen", api_key="prompt", validate=True)

In [ ]:
# @title 🗂️ Set the dataset path
import pandas as pd


# Configuration for displaying pandas dataframe
pd.set_option('display.max_colwidth', 500)

# Constants
MODEL = 'meta-llama/Llama-2-7b-chat-hf' # @param {type:"string"}
NUM_RECORDS = 250 # @param {type:"integer"}
DATASET = 'gsm8k' # @param ["gsm8k"]

We'll use the [GSM8K](https://paperswithcode.com/dataset/gsm8k) (Grade School Math) dataset of 8k real questions to query the LLM. These are problems that take between 2 and 8 steps to solve, primarily performing a sequence of elementary calculations using arithmetic to reach the final answer, creating a good benchmark for assessing LLMs ability to solve everyday real-world multi-step problems.

## 📊 Prepare your prompt data

- Load the GSM8k dataset from Huggingface Datasets.

- Use a prompt template to load the questions from GSM8k into the expected Llama2 prompt format.

In [ ]:
import textwrap

from datasets import load_dataset

# Function to format prompt according to Llama2 expected chat instruction format
def format_prompt(prompt: str) -> str:
    llama_template = textwrap.dedent(f"""\
    <s>[INST] <<SYS>>You provide just the answer you are asked for with no preamble. Do not repeat the question. Be succinct.<</SYS>>

    {prompt} [/INST]
    """)

    return llama_template

# Load dataset with the 'main' configuration and get the first NUM_RECORDS questions
dataset = load_dataset('gsm8k', 'main')
questions = dataset['train']['question'][:NUM_RECORDS]

# Add the Llama2 instruction format to each prompt
formatted_prompts = [format_prompt(q) for q in questions]

# Convert the instructions to a dataframe format
questions = pd.DataFrame(data={'text': questions})
instructions = pd.DataFrame(data={'text': formatted_prompts})

# Print a random sample question and formatted instruction
random_idx = instructions.sample(n=1).index[0]
print(f"Random question:\n```{questions.loc[random_idx]['text']}```\n\n")
print(f"Instruction:\n```{instructions.loc[random_idx]['text']}```")

## 🎛️ Initialize the LLM

- We use the natural language [base config](https://github.com/gretelai/gretel-blueprints/blob/main/config_templates/gretel/synthetics/natural-language.yml) by setting `base_config="natural-language"`.

- As the goal is zero/few-shot prompting (i.e., no fine tuning), set `data_source=None`.

In [ ]:
pretrained = gretel.submit_train(
    base_config="natural-language",
    pretrained_model=MODEL,
    data_source=None
    )

## 🤖 Generate synthetic data

- You can pass any of Gretel GPT's [`generate` parameters](https://docs.gretel.ai/reference/synthetics/models/gretel-gpt#data-generation) as keyword arguments in the `submit_generate` method.



In [ ]:
generated = gretel.submit_generate(
    pretrained.model_id,
    maximum_text_length=250,
    seed_data=instructions,
    temperature=0.8
)

In [ ]:
results = generated.synthetic_data

# Combine original questions and results into a single DataFrame
df = pd.DataFrame({'question': questions['text'], 'answer': results['text']})
df.to_csv('questions_and_answers.csv', index=False)
df